# Pertemuan 6 — Data Visualization (Google Colab)

Notebook ini memandu *step-by-step* membuat **3 grafik** dari dataset `m6_visualization.csv`:
1) Bar: Rata-rata `machine_speed` per `shift`
2) Scatter: Hubungan `temperature_c` vs `downtime_min`
3) Pie: Proporsi `is_defect` (0/1)

> Catatan kelas:
> - Gunakan **matplotlib** (tanpa seaborn),
> - **Satu gambar per plot** (tanpa subplots),
> - **Tidak mengatur warna khusus** (pakai default).

## 0) Import paket
Jalankan sel ini terlebih dahulu.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

## 1) Memuat dataset
Pilih **salah satu** cara:
- **A. Upload manual** file `m6_visualization.csv`
- **B. Google Drive** (pastikan file tersedia di Drive Anda)

> Jalankan sel A **atau** B saja. Kalau keduanya jalan, yang terakhir akan dipakai.

### 1A) Upload manual (Colab)

In [ ]:
DATA_PATH = None
try:
    from google.colab import files  # type: ignore
    print("Silakan pilih file m6_visualization.csv untuk di-upload...")
    uploaded = files.upload()
    DATA_PATH = list(uploaded.keys())[0]
    print("File terunggah:", DATA_PATH)
except Exception as e:
    print("Upload Colab tidak tersedia. Lewati langkah ini jika tidak memakai Colab.")

### 1B) Google Drive (opsional)

In [ ]:
# Ganti path sesuai lokasi file Anda di Google Drive
GDRIVE_PATH = "/content/drive/MyDrive/m6_visualization.csv"
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
    DATA_PATH = GDRIVE_PATH
    print("Drive ter-mount. DATA_PATH =", DATA_PATH)
except Exception as e:
    print("Mount Drive tidak tersedia. Jika tidak memakai Colab, lewati langkah ini.")

### 1C) Default (jika file ada di working directory)
Jika tidak memakai 1A/1B, set manual lokasi file (misal jika file sudah ada di folder kerja).

In [ ]:
if DATA_PATH is None:
    DATA_PATH = "m6_visualization.csv"  # asumsikan file ada di working directory
print("Menggunakan DATA_PATH =", DATA_PATH)

## 2) Load & cek data
- Memuat CSV
- Cek dimensi, beberapa baris pertama, tipe kolom, dan *missing values*

In [ ]:
# Load data
df = pd.read_csv(DATA_PATH)
print('Dimensi data:', df.shape)
display(df.head())

# Info struktur
print('\nInfo kolom:')
print(df.info())

# Cek missing values (seharusnya 0 untuk dataset bersih sesi visualisasi)
print('\nJumlah nilai kosong per kolom:')
print(df.isna().sum())
assert df.isna().sum().sum() == 0, 'Dataset masih mengandung nilai kosong. Gunakan m6_visualization.csv yang sudah bersih.'

## 3) Statistik deskriptif singkat
Memberi gambaran data numerik dan kategorikal.

In [ ]:
display(df.describe(include='all'))

## 4) Visualisasi 1 — Bar Chart
**Tujuan**: Rata-rata `machine_speed` per `shift`.

**Langkah**:
1. Kelompokkan berdasarkan `shift`.
2. Hitung mean `machine_speed`.
3. Plot bar (satu gambar/plot).

In [ ]:
mean_speed = df.groupby('shift')['machine_speed'].mean().reindex(['pagi','siang','malam'])

plt.figure()
mean_speed.plot(kind='bar')
plt.title('Rata-rata Kecepatan Mesin per Shift')
plt.xlabel('Shift')
plt.ylabel('Machine Speed (unit/min)')
plt.tight_layout()
plt.show()

## 5) Visualisasi 2 — Scatter Plot
**Tujuan**: Melihat hubungan `temperature_c` (X) terhadap `downtime_min` (Y).

**Langkah**:
1. Ambil dua kolom tersebut.
2. Plot titik *scatter* (satu gambar/plot).

In [ ]:
                plt.figure()
plt.scatter(df['temperature_c'], df['downtime_min'], alpha=0.6)
plt.title('Hubungan Suhu dan Downtime')
plt.xlabel('Temperature (°C)')
plt.ylabel('Downtime (menit)')
plt.tight_layout()
plt.show()

## 6) Visualisasi 3 — Pie Chart
**Tujuan**: Mengetahui proporsi `is_defect` (0 = Tidak Cacat, 1 = Cacat).

**Langkah**:
1. Hitung frekuensi 0/1.
2. Plot *pie* (satu gambar/plot).

In [ ]:
                counts = df['is_defect'].value_counts().sort_index()  # urutkan 0 lalu 1
labels = ['Tidak Cacat (0)', 'Cacat (1)']

plt.figure()
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Proporsi Produk Cacat')
plt.tight_layout()
plt.show()

## 7) (Opsional) Simpan grafik ke file
Simpan tiga grafik sebagai PNG di folder `outputs/` dan unduh sebagai ZIP.

In [ ]:
                SAVE = True
if SAVE:
    import zipfile
    out_dir = 'outputs'
    os.makedirs(out_dir, exist_ok=True)

    # a) Bar
    plt.figure()
    mean_speed.plot(kind='bar')
    plt.title('Rata-rata Kecepatan Mesin per Shift')
    plt.xlabel('Shift')
    plt.ylabel('Machine Speed (unit/min)')
    plt.tight_layout()
    bar_path = os.path.join(out_dir, 'chart_bar_mean_speed.png')
    plt.savefig(bar_path, dpi=200)
    plt.close()

    # b) Scatter
    plt.figure()
    plt.scatter(df['temperature_c'], df['downtime_min'], alpha=0.6)
    plt.title('Hubungan Suhu dan Downtime')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Downtime (menit)')
    plt.tight_layout()
    scatter_path = os.path.join(out_dir, 'chart_scatter_temp_downtime.png')
    plt.savefig(scatter_path, dpi=200)
    plt.close()

    # c) Pie
    plt.figure()
    plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90)
    plt.title('Proporsi Produk Cacat')
    plt.tight_layout()
    pie_path = os.path.join(out_dir, 'chart_pie_defect.png')
    plt.savefig(pie_path, dpi=200)
    plt.close()

    # Buat ZIP
    zip_path = 'outputs/figs_outputs.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(bar_path, arcname=os.path.basename(bar_path))
        zf.write(scatter_path, arcname=os.path.basename(scatter_path))
        zf.write(pie_path, arcname=os.path.basename(pie_path))

    print('Gambar disimpan di folder:', out_dir)
    print('ZIP tersedia di:', zip_path)

## 8) (Opsional) Catatan interpretasi cepat
- **Bar**: shift dengan rata-rata `machine_speed` tertinggi mengindikasikan efisiensi lebih baik.
- **Scatter**: cek pola kenaikan downtime di suhu tertentu dan identifikasi outlier.
- **Pie**: perbandingan proporsi cacat vs tidak cacat untuk indikator kualitas.